In [2]:
import pathlib
import os
import json
from tqdm import trange, tqdm
import pandas as pd

In [2]:
df = pd.read_csv('/mnt/NAS/datasets/LuCAS-Plus/LungSegmentation/LUNA16/candidates.csv')
df_v2 = pd.read_csv('/mnt/NAS/datasets/LuCAS-Plus/LungSegmentation/LUNA16/candidates_V2.csv')

In [3]:
df_v2

,seriesuid,coordX,coordY,coordZ,class
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,68.420000,-74.480000,-288.700000,0
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-95.209361,-91.809406,-377.426350,0
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-24.766755,-120.379294,-273.361539,0
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-63.080000,-65.740000,-344.240000,0
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,52.946688,-92.688873,-241.067872,0
...,...,...,...,...,...
754970,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-33.400000,-64.200000,-115.560000,0
754971,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,56.236359,70.352400,-203.446236,0
754972,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-97.104221,55.738289,-203.879785,0
754973,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-65.470000,59.670000,-136.370000,0


In [39]:
dff = df[df['seriesuid']=='1.3.6.1.4.1.14519.5.2.1.6279.6001.241717018262666382493757419144']
dfff = dff[dff['class']==0]
dfff

,seriesuid,coordX,coordY,coordZ,class
259316,1.3.6.1.4.1.14519.5.2.1.6279.6001.241717018262...,109.457119,36.832119,-226.585811,0
259317,1.3.6.1.4.1.14519.5.2.1.6279.6001.241717018262...,-55.721636,-12.932702,-185.411885,0
259318,1.3.6.1.4.1.14519.5.2.1.6279.6001.241717018262...,90.603620,45.880489,-209.032944,0
259319,1.3.6.1.4.1.14519.5.2.1.6279.6001.241717018262...,-47.488666,52.716185,-126.389926,0
259320,1.3.6.1.4.1.14519.5.2.1.6279.6001.241717018262...,57.122174,21.583733,-143.151923,0
...,...,...,...,...,...
259628,1.3.6.1.4.1.14519.5.2.1.6279.6001.241717018262...,39.216640,19.473770,-232.018578,0
259629,1.3.6.1.4.1.14519.5.2.1.6279.6001.241717018262...,40.915812,39.142886,-176.671181,0
259630,1.3.6.1.4.1.14519.5.2.1.6279.6001.241717018262...,38.902647,3.769070,-107.976673,0
259631,1.3.6.1.4.1.14519.5.2.1.6279.6001.241717018262...,-53.492719,20.083046,-151.633703,0


In [28]:
### Series UID check

series_uids = list(set(df['seriesuid']))
loss_list = []
for uid in tqdm(series_uids):
    if not os.path.exists(os.path.join('/data/dk/LUNA16_crops/1', uid)) \
        and not os.path.exists(os.path.join('/data/dk/LUNA16_crops/0', uid)):
        loss_list.append(uid)

print(len(loss_list))
with open('/data/dk/LUNA16_crops/remain_series_uids.json', 'w') as file:
    json.dump(loss_list, file)



100%|██████████| 888/888 [00:08<00:00, 100.30it/s]

0


In [47]:
cnt = 0
loss_list = []
for i in trange(len(df)):
    cur_df = df.iloc[i]
    cur_path = os.path.join('/data/dk/LUNA16_crops', str(cur_df['class']), cur_df['seriesuid'],
                            '_'.join([str(int(cur_df['coordZ'])), 
                                      str(int(cur_df['coordY'])), 
                                      str(int(cur_df['coordX']))])+'_1.0.npy')
    if not os.path.exists(cur_path):
        # print(cur_path)
        if cur_df['seriesuid'] not in loss_list:
            loss_list.append(cur_df['seriesuid'])
        cnt += 1
print(f"missed - series {len(loss_list)}, cnt {cnt}")
# with open('/data/dk/LUNA16_crops/remain_series_uids_2.json', 'w') as file:
#     json.dump(loss_list, file)

100%|██████████| 551065/551065 [02:45<00:00, 3326.13it/s]

missed - series 0, cnt 0


## Split candidates merged

In [30]:
df = pd.read_csv('/mnt/NAS/datasets/LuCAS-Plus/LungSegmentation/LUNA16/candidates_merged.csv')
df = df.drop([df.columns[0]], axis=1)
display(df)

,seriesuid,coordX,coordY,coordZ,class,diameter_mm
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,68.420000,-74.480000,-288.700000,0,NaN
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-95.209361,-91.809406,-377.426350,0,NaN
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-24.766755,-120.379294,-273.361539,0,NaN
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-63.080000,-65.740000,-344.240000,0,NaN
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,52.946688,-92.688873,-241.067872,0,NaN
...,...,...,...,...,...,...
754970,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-29.720000,24.620000,-156.010000,1,NaN
754971,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,42.895122,74.251037,-200.625978,1,4.277203
754972,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-28.685000,32.690000,-164.835000,1,NaN
754973,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-23.719297,31.714489,-156.172233,1,23.802913


In [31]:
pos_list = df[df['class']==1]
neg_list = df[df['class']==0]
print(len(pos_list), len(neg_list))

1557 753418


In [32]:
pos_cnt, neg_cnt = 0, 0
split_list = []
for i in trange(len(df)):
    if df['class'][i]==1:
        pos_cnt += 1
        if pos_cnt < len(pos_list) * 0.7:
            split_list.append('train')
        elif pos_cnt < len(pos_list) * 0.85:
            split_list.append('valid')
        else:
            split_list.append('test')
    elif df['class'][i]==0:
        neg_cnt += 1
        if neg_cnt < len(neg_list) * 0.7:
            split_list.append('train')
        elif neg_cnt < len(neg_list) * 0.85:
            split_list.append('valid')
        else:
            split_list.append('test')
df.insert(6,'Split', split_list)
display(df)
    
    

100%|██████████| 754975/754975 [00:08<00:00, 92433.80it/s]


,seriesuid,coordX,coordY,coordZ,class,diameter_mm,Split
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,68.420000,-74.480000,-288.700000,0,NaN,train
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-95.209361,-91.809406,-377.426350,0,NaN,train
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-24.766755,-120.379294,-273.361539,0,NaN,train
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-63.080000,-65.740000,-344.240000,0,NaN,train
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,52.946688,-92.688873,-241.067872,0,NaN,train
...,...,...,...,...,...,...,...
754970,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-29.720000,24.620000,-156.010000,1,NaN,test
754971,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,42.895122,74.251037,-200.625978,1,4.277203,test
754972,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-28.685000,32.690000,-164.835000,1,NaN,test
754973,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-23.719297,31.714489,-156.172233,1,23.802913,test


In [33]:
pos_train = df[(df['class']==1) & (df['Split']=='train')]
pos_valid = df[(df['class']==1) & (df['Split']=='valid')]
pos_test = df[(df['class']==1) & (df['Split']=='test')]

neg_train = df[(df['class']==0) & (df['Split']=='train')]
neg_valid = df[(df['class']==0) & (df['Split']=='valid')]
neg_test = df[(df['class']==0) & (df['Split']=='test')]

print(len(pos_train), len(pos_valid), len(pos_test))
print(len(neg_train), len(neg_valid), len(neg_test))

1089 234 234
527392 113013 113013


In [2]:
df.to_csv('/mnt/NAS/datasets/LuCAS-Plus/LungSegmentation/LUNA16/candidates_merged_split.csv')

NameError: name 'df' is not defined

In [24]:
df_merged = pd.read_csv('/mnt/NAS/datasets/LuCAS-Plus/LungSegmentation/LUNA16/candidates_merged_split.csv')
train = df_merged[(df_merged['Split']=='train')]
pos_valid = df_merged[(df_merged['class']==1) & (df_merged['Split']=='valid')]
pos_test = df_merged[(df_merged['class']==1) & (df_merged['Split']=='test')]
neg_valid = df_merged[(df_merged['class']==0) & (df_merged['Split']=='valid')]
neg_test = df_merged[(df_merged['class']==0) & (df_merged['Split']=='test')]

In [18]:
train = train.reset_index(drop=True).drop(train.columns[0], axis=1)
train.to_csv('train.csv')
pos_valid = pos_valid.reset_index(drop=True).drop(pos_valid.columns[0], axis=1)
pos_valid.to_csv('valid_pos.csv')
pos_test = pos_test.reset_index(drop=True).drop(pos_test.columns[0], axis=1)
pos_test.to_csv('test_pos.csv')
neg_valid = neg_valid.reset_index(drop=True).drop(neg_valid.columns[0], axis=1)
neg_valid.to_csv('valid_neg.csv')
neg_test = neg_test.reset_index(drop=True).drop(neg_test.columns[0], axis=1)
neg_test.to_csv('test_neg.csv')

In [19]:
neg_test

,seriesuid,coordX,coordY,coordZ,class,diameter_mm,Split
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.566816709786...,65.065000,66.155000,-250.880000,0,NaN,test
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.566816709786...,-113.584000,36.313000,-252.270000,0,NaN,test
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.566816709786...,-49.570000,-10.530000,-136.200000,0,NaN,test
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.566816709786...,55.555000,58.185000,-108.395000,0,NaN,test
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.566816709786...,-67.050000,90.000000,-226.000000,0,NaN,test
...,...,...,...,...,...,...,...
113008,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-33.400000,-64.200000,-115.560000,0,NaN,test
113009,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,56.236359,70.352400,-203.446236,0,NaN,test
113010,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-97.104221,55.738289,-203.879785,0,NaN,test
113011,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-65.470000,59.670000,-136.370000,0,NaN,test


In [23]:
import random
a = [1,2,3,4,5]
b = random.sample(a, 1)
print(b)

[1]


## Data existence check

In [37]:
import numpy as np
data_path = '/data/dk/LUNA16_crops'
cnt = 0
missed_indexes = {}
for idx in trange(len(df_merged)):
    cur_path = os.path.join(data_path, str(df_merged['class'][idx]), df_merged['seriesuid'][idx],
                                '_'.join([str(int(df_merged['coordZ'][idx])),str(int(df_merged['coordY'][idx])),
                                          str(int(df_merged['coordX'][idx])), '1.0'])+'.npy')
    if not os.path.exists(cur_path):
        # print(cur_path)
        # input('enter')
        if df_merged['seriesuid'][idx] in missed_indexes.keys():
            missed_indexes[df_merged['seriesuid'][idx]].append(idx)
        else:
            missed_indexes[df_merged['seriesuid'][idx]] = [idx]
        cnt += 1
print(cnt)
# with open('missed_indexes.json', 'w') as file:
#     json.dump(missed_indexes, file)
    

100%|██████████| 754975/754975 [00:25<00:00, 29174.46it/s]


284338


In [39]:
missed_dict = json.load(open('missed_indexes.json', 'r'))
len(missed_dict)
i_cnt = 0
for m, i in missed_dict.items():
    i_cnt += len(i)
print(i_cnt)

284338


In [27]:
series_df = df_merged[df_merged['seriesuid']=='1.3.6.1.4.1.14519.5.2.1.6279.6001.288701997968615460794642979503']
display(series_df, )

,Unnamed: 0,seriesuid,coordX,coordY,coordZ,class,diameter_mm,Split
460097,460097,1.3.6.1.4.1.14519.5.2.1.6279.6001.288701997968...,-100.240000,40.530000,1448.600000,0,NaN,train
460098,460098,1.3.6.1.4.1.14519.5.2.1.6279.6001.288701997968...,71.160000,108.120000,1522.000000,0,NaN,train
460099,460099,1.3.6.1.4.1.14519.5.2.1.6279.6001.288701997968...,90.133206,61.815735,1578.082941,0,NaN,train
460100,460100,1.3.6.1.4.1.14519.5.2.1.6279.6001.288701997968...,-111.570000,13.860000,1415.750000,0,NaN,train
460101,460101,1.3.6.1.4.1.14519.5.2.1.6279.6001.288701997968...,-101.546743,66.896689,1527.765878,0,NaN,train
...,...,...,...,...,...,...,...,...
461702,461702,1.3.6.1.4.1.14519.5.2.1.6279.6001.288701997968...,-61.023445,-43.022361,1486.588194,0,NaN,train
461703,461703,1.3.6.1.4.1.14519.5.2.1.6279.6001.288701997968...,55.410000,21.620000,1478.000000,0,NaN,train
754422,754422,1.3.6.1.4.1.14519.5.2.1.6279.6001.288701997968...,-107.141159,81.899083,1433.617467,1,5.611908,train
754423,754423,1.3.6.1.4.1.14519.5.2.1.6279.6001.288701997968...,-24.035925,67.030000,1569.935000,1,5.407599,train


In [28]:
series_df.iloc[:10]

,Unnamed: 0,seriesuid,coordX,coordY,coordZ,class,diameter_mm,Split
460097,460097,1.3.6.1.4.1.14519.5.2.1.6279.6001.288701997968...,-100.240000,40.530000,1448.600000,0,NaN,train
460098,460098,1.3.6.1.4.1.14519.5.2.1.6279.6001.288701997968...,71.160000,108.120000,1522.000000,0,NaN,train
460099,460099,1.3.6.1.4.1.14519.5.2.1.6279.6001.288701997968...,90.133206,61.815735,1578.082941,0,NaN,train
460100,460100,1.3.6.1.4.1.14519.5.2.1.6279.6001.288701997968...,-111.570000,13.860000,1415.750000,0,NaN,train
460101,460101,1.3.6.1.4.1.14519.5.2.1.6279.6001.288701997968...,-101.546743,66.896689,1527.765878,0,NaN,train
460102,460102,1.3.6.1.4.1.14519.5.2.1.6279.6001.288701997968...,-83.530000,107.780000,1419.970000,0,NaN,train
460103,460103,1.3.6.1.4.1.14519.5.2.1.6279.6001.288701997968...,79.830478,49.532787,1457.851647,0,NaN,train
460104,460104,1.3.6.1.4.1.14519.5.2.1.6279.6001.288701997968...,77.562053,82.817028,1431.800926,0,NaN,train
460105,460105,1.3.6.1.4.1.14519.5.2.1.6279.6001.288701997968...,-88.830000,-30.695000,1447.775000,0,NaN,train
460106,460106,1.3.6.1.4.1.14519.5.2.1.6279.6001.288701997968...,68.535000,-49.830000,1452.875000,0,NaN,train


In [46]:
import numpy as np
for idx in trange(len(df_merged)):
    cur_path = os.path.join(data_path, str(df_merged['class'][idx]), df_merged['seriesuid'][idx],
                                '_'.join([str(int(df_merged['coordZ'][idx])),str(int(df_merged['coordY'][idx])),
                                          str(int(df_merged['coordX'][idx])), '1.0'])+'.npy')
    arr = np.load(cur_path)
    print(arr.dtype)
    print(arr.max(), arr.min())
    n_arr = (arr*255.).astype(np.uint8)
    print(n_arr.dtype)
    print(n_arr.max(), n_arr.min())
    break

  0%|          | 0/754975 [00:00<?, ?it/s]

float64
1.0764036830054433 -0.024329230673400957
uint8
255 0
